# CHAPTER 13
# Introduction to Modeling Libraries in Python
- In this chapter, we review some features of pandas that may be helpful when you’re crossing back and forth between **data wrangling** with **pandas** and **model fitting** and **scoring**. 
- Short introductions to two popular modeling toolkits, **statsmodels** and **scikit-learn**.

## Interfacing Between pandas and Model Code
- A common workflow for model development is to use **pandas** for **data loading and cleaning** before switching over to a modeling library to build the model itself. 
- An important part of the model development process is called **feature engineering** in machine learning. 
- This can describe any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context. 
- The **data aggregation and GroupBy** tools we have explored in this book are used often in a feature engineering context.
- The point of contact between **pandas** and other analysis libraries is usually **NumPy arrays**. 
- To turn a DataFrame into a NumPy array, use the **.values property**.
- The **.values** attribute is intended to be used when your data is homogeneous — for example, all numeric types. 
- If you have heterogeneous data, the result will be an **ndarray** of Python objects.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Example DataFrame
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
# Select the columns
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
# Select the columns values
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
# DataFrame example
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [6]:
# Select the values for heterogeneous data
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [7]:
# Use a subset of the column
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

- Some libraries have native support for pandas and do some of this work for you automatically: converting to NumPy from DataFrame and attaching model parameter names to the columns of output tables or Series. 
- In other cases, you will have to perform this “metadata management” manually.

In [8]:
# DataFrame example
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [9]:
# Create dummy variables, drop the 'category' column, and then join the result
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## Creating Model Descriptions with Patsy
- **Patsy** is a Python library for describing **statistical models** (especially linear models) with a small string-based *formula syntax*.
- **Patsy** is well supported for specifying **linear models in statsmodels**. 
- **Patsy’s** formulas are a special string syntax that looks like:
        y ~ x0 + x1
- The syntax a + b does not mean to add a to b, but rather that these are terms in the **design matrix** created for the model. 
- The **patsy.dmatrices** function takes a formula string along with a dataset (which can be a DataFrame or a dict of arrays) and produces **design matrices** for a linear model.

In [10]:
# Example DataFrame
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [11]:
import patsy

# Input for patsy
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [12]:
# Display y
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [13]:
# Display X
X

# These Patsy DesignMatrix instances are NumPy ndarrays with additional metadata

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [14]:
# The Patsy objects can be passed directly into algorithms like numpy.linalg.lstsq,
# which performs an ordinary least squares regression
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

In [15]:
# The model metadata is retained in the design_info attribute, so you can reattach the
# model column names to the fitted coefficients to obtain a Series
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformations in Patsy Formulas
- You can mix Python code into your Patsy formulas
- When evaluating the formula the library will try to find the functions you use in the enclosing scope

In [16]:
# Example Patsy formulas
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [17]:
# Some commonly used variable transformations include standardizing (to mean 0 and
# variance 1) and centering (subtracting the mean). Patsy has built-in functions for this purpose

y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [18]:
# The patsy.build_design_matrices function can apply transformations to new outof-sample 
# data using the saved information from the original in-sample dataset

new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

### Categorical Data and Patsy
- Non-numeric data can be transformed for a model design matrix in many different ways.
- When you use non-numeric terms in a Patsy formula, they are converted to dummy variables by default.

In [19]:
# Example for categorical data
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7] })

y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

## Introduction to statsmodels
- **statsmodels** is a Python library for fitting many kinds of statistical models, performing statistical tests, and data exploration and visualization. 
- **statsmodels** contains more “classical” frequentist statistical methods, while Bayesian methods and machine learning models are found in other libraries.
- Some kinds of models found in statsmodels include:
    - Linear models, generalized linear models, and robust linear models
    - Linear mixed effects models
    - Analysis of variance (ANOVA) methods
    - Time series processes and state space models
    - Generalized method of moments

### Estimating Linear Models
- There are several kinds of **linear regression** models in statsmodels, from the more basic (e.g., ordinary least squares) to more complex (e.g., iteratively reweighted least squares).
- Linear models in statsmodels have two different main interfaces: array-based and formula-based. 
- These are accessed through these API module imports.

In [20]:
# Import modules
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [21]:
# Generate a linear model from some random data

# dnorm is a helper function for generating normally distributed data 
# with a particular mean and variance
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# For reproducibility
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]

eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [22]:
# Check X array
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [23]:
# Check y array
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [24]:
# A linear model is generally fitted with an intercept term as we saw before with Patsy
# The sm.add_constant function can add an intercept column to an existing matrix
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [25]:
# sm.OLS class can fit an ordinary least squares linear regression
model = sm.OLS(y, X)

In [26]:
# The model’s fit method returns a regression results object containing estimated
# model parameters and other diagnostics
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Wed, 24 Mar 2021   Prob (F-statistic):                    7.44e-12
Time:                        18:13:13   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## Introduction to scikit-learn
- **scikit-learn** is one of the most widely used and trusted general-purpose Python machine learning toolkits. 
- It contains a broad selection of standard supervised and unsupervised machine learning methods with tools for model selection and evaluation, data transformation, data loading, and model persistence. 
- These models can be used for classification, clustering, prediction, and other common tasks.

In [27]:
# Example Kaggle Titanic dataset
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [ ]:
# Libraries like statsmodels and scikit-learn generally cannot be fed missing data, so we
# look at the columns to see if there are any that contain missing data
train.isnull().sum()

In [ ]:
# Missing data for test dataset
test.isnull().sum()

- In statistics and machine learning examples like this one, a typical task is to predict whether a passenger would survive based on features in the data. 
- A model is fitted on a training dataset and then evaluated on an out-of-sample testing dataset.
- I would like to use Age as a predictor, but it has missing data. 
- There are a number of ways to do missing data imputation, but I will do a simple one and use the median of the training dataset to fill the nulls in both tables.

In [ ]:
# Input missing values
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [ ]:
# Add a column IsFemale as an encoded version of the 'Sex' column
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [ ]:
# Then we decide on some model variables and create NumPy arrays
predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].values
X_test = test[predictors].values

y_train = train['Survived'].values

In [ ]:
# Check X_train
X_train[:5]

In [ ]:
# We use the LogisticRegression model from scikit-learn and create a model instance
from sklearn.linear_model import LogisticRegression

# We can fit this model to the training data using the model’s fit method
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# We can form predictions for the test dataset using model.predict
y_predict = model.predict(X_test)
y_predict[:10]

- In practice, there are often many additional layers of complexity in model training.
- Many models have parameters that can be **tuned**, and there are techniques such as **cross-validation** that can be used for parameter tuning to avoid overfitting to the training data. 
- This can often yield better predictive performance or robustness on new data.
- **Cross-validation** works by splitting the training data to simulate out-of-sample prediction. 
- Based on a model accuracy score like mean squared error, one can perform a **grid search** on model parameters. 
- ome models, like logistic regression, have estimator classes with built-in cross-validation. 
- For example, the **LogisticRegressionCV** class can be used with a parameter indicating how fine-grained of a grid search to do on the model regularization parameter C.

In [ ]:
# Perform parameter tuning with grid search
from sklearn.linear_model import LogisticRegressionCV
model_cv = LogisticRegressionCV(Cs=10)
model_cv.fit(X_train, y_train)

- To do cross-validation by hand, you can use the **cross_val_score** helper function, which handles the data splitting process. 
- The default scoring metric is model-dependent, but it is possible to choose an explicit scoring function. 
- Cross-validated models take longer to train, but can often yield better model performance.

In [ ]:
# Cross-validate our model with four non-overlapping splits of the training data

from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)

scores = cross_val_score(model, X_train, y_train, cv=4)
scores

## Example - Kaggle Titanic Dataset
- https://www.kaggle.com/alexandraneagu/titanic-predicting-survival-of-passengers